In [ ]:
# This is very rough draft but good to see it as it was developed
# options for name of package -> #1 pgnalyzer #2 pandoCal # 

In [ ]:
# Importing important packages
%matplotlib widget
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.stats import pearsonr
from astropy.coordinates import get_sun
from astropy.time import Time
from tqdm import tqdm
from scipy.optimize import least_squares
import cv2

In [ ]:
# Read PGN data file 
# returns cc, used_darks, and param_idx of headers
# Import important libraries

##### Creating important functions
# Creating a function that will make UTC Dates compatible with np.datetime64() object
def upDate(date: str) -> str:
    """
    Take in PGN YYYYMMDDTHHMMSS.MSZ and convert it into a string"
    
    YYYYMMDDTHHMMSS.MSZ  -->  'YYYY-MM-DDTHH:MM:SS.MS'"
    
    Not immediately converted into np.datetime64 to conserve as label.
    """
    return f"{date[:4]}-{date[4:6]}-{date[6:8]}T{date[9:11]}:{date[11:13]}:{date[13:-1]}"

# Creating a function that will locate the start of the Pandora Instrument's data
def findDataStart(in_path: str) -> str:
    with open(in_path, encoding = "latin1") as file:
        lines = file.readlines()
        intro_line_passed = False
        header_line_passed = False
        for i, line in enumerate(lines): 
            if "-------" in line and intro_line_passed:
                header_line_passed = True
            elif "-------" in line:
                intro_line_passed = True
            elif header_line_passed and intro_line_passed:
                intro_line_passed, header_line_passed = False, False
                return line[:2]

# Creating a function that uses findDataStart to located the final characters of the introduction("metadata") and header            
def findIntroHeader(in_path: str, data_st_chars) -> list[str, str]:
    with open(in_path, encoding = "latin1") as file:
        intro_end = file.seek(file.read().find("-\nColumn"), 0) + len("-\n")
    with open(in_path, encoding = "latin1") as file:
        header_end = file.seek(file.read().find(f"-\n{data_st_chars}"), 0) + len("-\n")
    char_dif = header_end - intro_end
    return intro_end, char_dif

# Creating a function that uses Introduction string to find Latitude and Longitude
def findLatLong(PGN_intro:str) -> list[str, str]:
    # Creating lat and long variables
    for line in PGN_intro.splitlines():
        if line.startswith("Location lat"):
            file_lat = line.split()[-1]
        elif line.startswith("Location long"):
            file_long = line.split()[-1]

    return file_lat, file_long

# Creating a function that uses Header string to find each unique column name for use as a header
def findHeaders(PGN_header: str) -> list:
    headers = []
    # Splitting all the lines in the txt file into a list to create header names
    for i, line in enumerate(tqdm(PGN_header.split("\n"), desc = "Creating column headers: ")):
        
        if i == len(PGN_header.split("\n")) - 4:
            for j in np.arange(1, int(line.split()[1].split("-")[1][:-1]) - int(line.split()[1].split("-")[0]) + 2):
                headers.append(
                    f"Pixel {j}" 
                )

        elif i == len(PGN_header.split("\n")) - 3:
            for j in np.arange(1, int(line.split()[1].split("-")[1][:-1]) - int(line.split()[1].split("-")[0]) + 2):
                headers.append(
                    f"Unc {j}" 
                )

        elif line.lower().startswith("column"):
            headers.append(
                " ".join(line.split()[2:7]) 
            )
    return headers

# Creating a function that uses the dataentry's Filterwheels as a check for dark measurements
def isDark(dataentry:str, param_index) -> bool:
    return dataentry.split()[param_index["Position of filterwheel #2, 0=filterwheel"]] == "3"

# Creating a function that takes full dark and separates it to calculation-critical information
def simplifyDarks(dark_routine:list, param_index:dict) -> dict:
    simple_darks = {}
    for dark in dark_routine:
        date = dark[param_index["UT date and time for"]]
        vals = dark[param_index["Pixel 1"]:param_index["Unc 1"]]
        exp_time = dark[param_index["Integration time [ms]"]]
        scale = dark[param_index["Scale factor for data (to"]]
        simple_darks[date] = {
            "PIX"       : np.array(list(map(lambda val: float(val), vals))),
            "SCALE"     : float(scale),
            "EXP_TIME"  : float(exp_time)  
        }
    return simple_darks

# Creating a function that sifts through all kinds of routines and   
# creates a dictionary that contains all routines and measurements
def separateData(PGN_data: list, headers:list, param_index: dict, latlong: list) -> dict:
    # Iterating through the list of strings
    a_routines, c_routines = [], []
    d_routines, e_routines, f_routines = [], [], []
    h_routines, k_routines, l_routines = [], [], []
    m_routines, r_routines, s_routines = [], [], []
    t_routines, w_routines, z_routines = [], [], []
    bad_data, unfiltered_data = [], []
    sun_darks_routine, moon_darks_routine = [], []
    sun_routine_labels = []
    for raw_data in tqdm(PGN_data, desc = "Sifting through all data: "):
        if raw_data.startswith("F"):
            f_routines.append(raw_data.split())
        elif raw_data.startswith("T"):
            t_routines.append(raw_data.split())
        elif raw_data.startswith("Z"):
            z_routines.append(raw_data.split())
        elif raw_data.startswith("R"):
            r_routines.append(raw_data.split())
        elif raw_data.startswith("W"):
            w_routines.append(raw_data.split())
        elif raw_data.startswith("L"):
            l_routines.append(raw_data.split())
        elif raw_data.startswith("K"):
            k_routines.append(raw_data.split())
        elif raw_data.startswith("H"):
            h_routines.append(raw_data.split())
        elif raw_data.startswith("E"):
            e_routines.append(raw_data.split())
        elif raw_data.startswith("D"):
            d_routines.append(raw_data.split())
        elif raw_data.startswith("A"):
            a_routines.append(raw_data.split())
        elif raw_data.startswith("C"):
            c_routines.append(raw_data.split())
        elif len(raw_data.split()) < 30: # Most L0 non-data dataentries cap at 24-26 characters (via LiftBlick manual) 
            bad_data.append(raw_data.split())
        elif raw_data.startswith("S"):
            if isDark(raw_data, param_index):
                sun_darks_routine.append(raw_data.split())
            else: 
                s_routines.append(raw_data.split())
                sun_routine_labels.append(raw_data.split()[param_index["Two letter code of measurement"]])
        elif raw_data.startswith("M"):
            if isDark(raw_data, param_index):
                moon_darks_routine.append(raw_data.split())
            else: 
                m_routines.append(raw_data.split())
        else:
            unfiltered_data.append(raw_data.split())

    sun_darks = simplifyDarks(sun_darks_routine, param_index)
    moon_darks = simplifyDarks(moon_darks_routine, param_index)
    
    sun_data = pd.DataFrame(s_routines, columns = headers)
    moon_data = pd.DataFrame(m_routines, columns = headers)

    routines = {
        "a" : a_routines, "c" : c_routines, "z" : z_routines,
        "d" : d_routines, "e" : e_routines, "f" : f_routines,
        "h" : h_routines, "k" : k_routines, "l" : l_routines,
        "r" : r_routines, "t" : t_routines, "w" : w_routines
    }


    return {
        "sun_data"  : sun_data,
        "moon_data" : moon_data,
        "routines"  : routines,
        "latlong"   : latlong,
        "param_idx" : param_index,
        "simple_sun_darks"  : sun_darks,
        "simple_moon_darks" : moon_darks,
        "sun_routines" : sun_routine_labels
    }

# Creating a function that opens a PGN L0 file and returns a python dictionary of file contents
# This function is a combination of all previous functions
def readPGN(input_file_path: str) -> dict:
    
    data_start_chars = findDataStart(input_file_path)
    intro_end_char, header_end_char = findIntroHeader(input_file_path, data_start_chars)
    
    with open(input_file_path, encoding = "latin1") as file:
        
        info_intro  = file.read(intro_end_char)
        info_header = file.read(header_end_char)
        info_data = file.readlines()

        latitude, longitude = findLatLong(info_intro)
        headers = findHeaders(info_header)
        param_idx = dict(zip(headers, np.arange(0, len(headers))))
                
        return separateData(info_data, headers, param_idx, [float(latitude), float(longitude)])

# Creating a function that selected the dark closest to a sun measurment in time and exposure_time        
def findBestDark(target_date: str, exp_time: float, dark_entries: dict, param_index: dict) -> dict:
    target_date = np.datetime64(target_date)
    min_date = (target_date - np.timedelta64(5, "m")).astype(np.datetime64)
    max_date = (target_date + np.timedelta64(5, "m")).astype(np.datetime64)

    date_search = [
        date for date in dark_entries 
        if (np.datetime64(upDate(date)) - min_date).astype(float) > 0 and 
        (np.datetime64(upDate(date)) - max_date).astype(float) < 0
    ]

    diffs = []
    for i, date in enumerate(date_search): 
        diffs.append(np.abs(dark_entries[date]["EXP_TIME"] - exp_time))
    closest_date = date_search[np.argmin(diffs)]

    return np.array(list(map(lambda pixel: float(pixel), dark_entries[closest_date]["PIX"]))), float(dark_entries[closest_date]["SCALE"]), closest_date    

# A function that brings all previous functions together to read the PGN file
def get_important_info(in_path: str, dark_corrected = False):
    # Loading data
    file_info = readPGN(in_path)

    # Dark Correction and Temporal Correction 
    sun_data = file_info["sun_data"].__array__()            # pd.Dataframe(dataentries, measurements) -> np.ndarray()  //  Easier to manage
    sun_darks = file_info["simple_sun_darks"]               # list of dict{"date" : dict{"PIX" : list(pixel vals), "SCALE" : float(scale), "EXP_TIME" : float(integration time)}
    param_idx = file_info["param_idx"]                      # dict{"header" : int(index)}
    latlong = file_info["latlong"]
    sun_routines = file_info["sun_routines"]

    sun_Fs = np.array(list(map(lambda measurement: float(measurement[param_idx["Scale factor for data (to"]]), sun_data)))
    sun_pix = np.array(list(map(lambda measurement: measurement[param_idx["Pixel 1"]:param_idx["Unc 1"]], sun_data)))

    sun_dates = np.array(list(map(
        lambda ut_date: upDate(ut_date), 
        list(map(lambda measurement: measurement[param_idx["UT date and time for"]], sun_data))
    )))
    sun_exp_times = np.array(list(map(lambda measurement: float(measurement[param_idx["Integration time [ms]"]]), sun_data)))

    ccs = []
    used_darks = []
    for i, date in enumerate(tqdm(sun_dates, desc = "Applying Dark Correction")):
        
        best_dark_pix, best_dark_F, best_dark_time = findBestDark(date, sun_exp_times[i], sun_darks, param_idx)
        
        if dark_corrected:
            ccs.append(
                ( (list(map(lambda pixel: float(pixel), sun_pix[i])) / sun_Fs[i]) - (best_dark_pix / best_dark_F) )
                / float(sun_exp_times[i])
            )

        else:
            ccs.append(np.array(list(map(lambda pixel: float(pixel), sun_pix[i]))))

        used_darks.append(
            best_dark_time
        )
    return ccs, sun_dates, used_darks, param_idx, latlong, sun_routines

# Set txt document information
# info_path = r"C:\GSFC\2025\05 May\Pandora2s1_GreenbeltMD_20250505_L0.txt" # 05-05-25
# ccs, cc_dates, used_darks, param_idx = correct_da_counts(info_path)

In [ ]:
# Creating functions

def get_model(input_file_path:str):
    data = pd.read_table(input_file_path, names = ["wavelength", "irradiance"], delimiter = " ", encoding = "latin1")
    return np.array(list(map(lambda val: float(val), data["wavelength"][data["irradiance"].values != 0]))), np.array(list(map(lambda val: float(val), data["irradiance"][data["irradiance"].values != 0])))

# ChatGPT IDL function -> python    
def ct2lst(jd, lon):
    """
    Convert Julian Date and longitude to Local Sidereal Time (LST).
    LST is needed to compute the observer's right ascension.
    """
    T = (jd - 2451545.0) / 36525.0  # Julian centuries since J2000.0
    GMST = 280.46061837 + 360.98564736629 * (jd - 2451545.0) + T**2 * (0.000387933 - T / 38710000.0)
    GMST = GMST % 360.0  # Normalize to [0, 360)
    LST = (GMST + lon) % 360.0  # Add longitude to get local sidereal time
    return LST / 15.0  # Convert degrees to hours

# Eddy IDL -> python
def get_solar_zenith_angle(date, lati, long):
    hour2radian = np.pi / 12 # there are 12 hours in pi radians 
    jd = Time(date, format = "datetime64").jd
    time = Time(jd, format = "jd")
    lst = ct2lst(jd, long) # This uses an AI generated conversion of IDL's ct2lst -> python

    ra_zen = lst * hour2radian
    dec_zen = np.deg2rad(lati)

    sun_pos = get_sun(time)
    ra_sun = sun_pos.ra.to_value(unit = "rad")
    dec_sun = sun_pos.dec.to_value(unit = "rad")

    th1 = np.pi/2 - dec_zen
    th2 = np.pi/2 - dec_sun
    ph1 = ra_zen
    ph2 = ra_sun

    sth1 = np.sin(th1)
    cth1 = np.cos(th1)
    sph1 = np.sin(ph1)
    cph1 = np.cos(ph1)

    sth2 = np.sin(th2)
    cth2 = np.cos(th2)
    sph2 = np.sin(ph2)
    cph2 = np.cos(ph2)

    x1 = sth1*cph1
    y1 = sth1*sph1
    z1 = cth1

    x2 = sth2*cph2
    y2 = sth2*sph2
    z2 = cth2

    return np.rad2deg(np.arccos(x1*x2 + y1*y2 + z1*z2))

# Stackexchange IDl function -> python 
def smooth(data, window):
    return np.convolve(data, np.ones(window)/window, mode='same')


In [ ]:
# Creating variables

chn = np.arange(
    start = 0, 
    stop = 2052, 
    step = 1
)

# spectral lines for calibration (Fraunhofer lines, Wikipedia, Wu)
w0 = np.array([
    822.7       , 759.4         , 722.2         , 686.7         , 656.3     , 627.7     , 589.3     , 527.0 , 
#   O2-Z        , O2-A          , H2O           , O2-B          , H-a       , O2-a      , Na        , Fe    ,    
    518.4       , 495.8         , 486.1         , 438.6         , 430.8     , 410.2     , 396.8     ,        
#   Mg(b1)      , Fe            , H-b           , Fe            , Ca/Fe     , H-d       , Ca+       ,       , 
	382.0       , 358.1         , 336.1         ,                                                       
#	Fe          , Fe            , TI+           ,               ,           ,           ,           ,       ,   
    # 302.1       , 299.4                                                                               
#	 Fe         , Ni            ,               ,               ,           ,           ,           ,       ,
])

nch = len(w0)
dw  = 10 # units of nanometers 

nn = 63 + 1
nch1 = 7
nch2 = nch


In [ ]:
# Eddy IDL -> python
def get_channel(
    wl_model_buff, 
    irr_model_buff, 
    wref_buff, 
    dw, 
    chn, 
    rad_buff, 
    a_buff, 
    b_buff, 
    cref0_buff
) -> int:
    cmax_buff = 0
    nn = 100 # Searching channel range +/- nn

    # Create search range in wavelengths
    idx_1_chn = np.where(abs(wl_model_buff - wref_buff) < dw)
    wl_m_buff = wl_model_buff[idx_1_chn]

    # Using min/max normalization on irradiance model
    want_model_buff = (
        (irr_model_buff[idx_1_chn] - np.min(irr_model_buff[idx_1_chn]))
        / (np.max(irr_model_buff[idx_1_chn]) - np.min(irr_model_buff[idx_1_chn]))
    )
    wavelength_range = np.arange(
        start = -nn, 
        stop = nn, 
        step = 1
    )
    # Initial fit to find channel for cref_buff
    for k_buff in wavelength_range:
        cref_buff = cref0_buff + 0.5*k_buff
        wl_buff = wref_buff + np.log((a_buff+b_buff*chn) / (a_buff+b_buff*cref_buff)) / b_buff

        idx_2_chn = np.where(np.abs(wl_buff - wref_buff) < dw)
        # Repeating min/max normalization for rad
        try:
            want_rad_buff = (
                (rad_buff[idx_2_chn] - np.min(rad_buff[idx_2_chn])) * 1
                / (np.max(rad_buff[idx_2_chn]) - np.min(rad_buff[idx_2_chn]))        
            )
            want_buff = np.interp(wl_m_buff, wl_buff[idx_2_chn], want_rad_buff)
            corr_buff = np.correlate(want_model_buff, want_buff, mode = "valid")
            
            if corr_buff > cmax_buff:
                cmax_buff = corr_buff
                cfit_buff = cref_buff
        
        except:
            print(f"ERROR: Iterating through wavelengths --> k: {k_buff}")
    try: 
        if not cfit_buff: print(cfit_buff)            
    except: 
        cfit_buff = 1000
        cmax_buff = 0.001
    return cfit_buff, cmax_buff

def fitSpectrum(
    model_input_path:str, 
    data_input_path:str,
    fs: int = 4, 
    icut: int = 5,
    showPlots: bool = True,
    wref: float = 430.8,     # nm
    cref0: int = 1150,      # first guess of channel number of this line
    rmin: float = 8e4,
):
    """
    This function replicates Dr. Wu's work
    """
    spec, xra = "s1", [300,600]

    wavelength_model, irradiance_model = get_model(model_input_path)
    irradiance_model_smooth = smooth(irradiance_model, icut)

    plt.close("all")
    # Plot 1
    if showPlots:
        fig, axes = plt.subplots(1, 1, figsize = (2.5 * fs, fs))
        axes.plot(
            wavelength_model,
            irradiance_model,
            color = "blue",
            linewidth = 0.5,
            label = "Model"
        )
        axes.plot(
            wavelength_model,
            irradiance_model_smooth,
            color = "orange",
            linewidth = 0.5,
            label = "Smoothed Model"
        )  
        axes.set_title("Model VS Raw Data")
        axes.set_xlabel("Wavelength (nm) // Channel")
        axes.set_ylabel("Intensity")    
        axes.legend(loc = "upper left", bbox_to_anchor = (0, -0.1))
        fig.tight_layout()
        plt.show()

    sun_data, sun_dates, used_darks, param_idx, latlong, sun_routines = get_important_info(data_input_path, dark_corrected = False) 

    so_obs = {
        "dates" : [sun_dates[i] for i in range(len(sun_data)) if sun_routines[i] == "SO"],
        "pix"   : [sun_data[i]  for i in range(len(sun_data)) if sun_routines[i] == "SO"]
    }

    sq_obs = {
        "dates" : [sun_dates[i] for i in range(len(sun_data)) if sun_routines[i] == "SQ"],
        "pix"   : [sun_data[i] for i, val in enumerate(sun_routines) if val == 'SQ']
    }

    ss_obs = {
        "dates" : [sun_dates[i] for i in range(len(sun_data)) if sun_routines[i] == "SS"],
        "pix"   : [sun_data[i] for i, val in enumerate(sun_routines) if val == 'SS']
    }

    su_obs = {
        "dates" : [sun_dates[i] for i in range(len(sun_data)) if sun_routines[i] == "SU"],
        "pix"   : [sun_data[i] for i, val in enumerate(sun_routines) if val == 'SU']
    }

    sb_obs = {
        "dates" : [sun_dates[i] for i in range(len(sun_data)) if sun_routines[i] == "SB"],
        "pix"   : [sun_data[i] for i, val in enumerate(sun_routines) if val == 'SB']
    }

    routes2use = sq_obs  ################################################################

    # Plot 2
    if showPlots:
        test_data_1 = sq_obs["pix"][10]
        test_data_2 = sq_obs["pix"][11]
        # test_rad = -test_data_1 + test_data_2
        test_rad = test_data_1 - test_data_2
        fig, axes = plt.subplots(1, 1, figsize = (2.5 * fs, fs))
        axes.plot(
            np.arange(len(test_data_1)).tolist(),
            test_rad,
            color = "blue",
            linewidth = 0.5,
            label = "Spec Data"
        )
        axes.legend(loc = "upper left", bbox_to_anchor = (1,1))
        axes.set_title(f"Spec Data: SZA = {get_solar_zenith_angle(np.datetime64(sun_dates[3]), latlong[0], latlong[1]):.3f}")
        axes.set_xticks(np.linspace(axes.get_xlim()[0], axes.get_xlim()[1], 50, dtype = int))
        axes.set_yticks(np.union1d(np.linspace(axes.get_ylim()[0], axes.get_ylim()[1], 5, dtype = int), [0]))
        xticks, yticks = axes.get_xticks(), axes.get_yticks()
        # axes.xaxis.set_ticklabels(axes.get_xticklabels(), rotation = 90)
        for lab in axes.get_xticklabels():
            lab.set_rotation(90)
        axes.set_xlabel("Channel")
        axes.set_ylabel("Rad")
        fig.suptitle("")
        fig.tight_layout()
        plt.show()

    if spec == "s1":
        # Following variables are for the first guess for s1
        a0      = 6.3
        b0      = 0.0013
        # a0 = 5.67
        # b0 = 0.00117
        # corr = [2.8358317e+19]
        # a0 = 5.103
        # b0 = 0.0010530000000000001
        # corr = [3.12896945e+19]
        # a0 = 5.05197
        # b0 = 0.0010951200000000002
        # corr = [3.18212983e+19]
        

    for i, timestamp in enumerate(routes2use["dates"]):
        if i != len(routes2use["dates"]) - 1:
            # if i in range(len(routes2use["dates"])):
            if i == 3:
                sza = get_solar_zenith_angle(np.datetime64(timestamp), latlong[0], latlong[1])

                rad = routes2use["pix"][i] - routes2use["pix"][i + 1]  
                if np.max(rad) > rmin:
                    cfit, _ = get_channel(
                        wl_model_buff    = wavelength_model, 
                        irr_model_buff   = irradiance_model_smooth, 
                        wref_buff   = wref, 
                        dw          = dw, 
                        chn         = chn, 
                        rad_buff         = rad, 
                        a_buff           = a0, 
                        b_buff           = b0, 
                        cref0_buff       = cref0
                    )
                    cref = cfit

                    idx = np.where(abs(wavelength_model - wref < dw))
                    want_model = (
                        (irradiance_model[idx] - np.min(irradiance_model[idx]))
                        / (np.max(irradiance_model[idx]) - np.min(irradiance_model[idx]))
                    ) 
                    smooth_model = (
                        (irradiance_model_smooth[idx] - np.min(irradiance_model_smooth[idx]))
                        / (np.max(irradiance_model_smooth[idx]) - np.min(irradiance_model_smooth[idx]))
                    ) 
                    # Plot 3
                    if showPlots:
                        fig, axes = plt.subplots(1, 1, figsize = (2.5 * fs, fs))
                        axes.plot(
                            wavelength_model[idx], 
                            want_model, 
                            linewidth = 0.5, 
                            color = "blue",
                            label = "Want Model"
                        )
                        axes.plot(
                            wavelength_model[idx],
                            smooth_model,
                            linewidth = 0.5,
                            color = "orange",
                            label = "Smooth Model"
                        )
                
                    afit, bfit = a0, b0
                    wl = wref + np.log((afit + bfit * chn) / (afit + bfit * cref)) / bfit
                    idx = np.where(abs(wl - wref < dw))
                    want_rad = (
                        (rad[idx] - np.min(rad[idx])) * 1
                        / (np.max(rad[idx]) - np.min(rad[idx]))
                    )
                    if showPlots:
                        axes.plot(
                            wl[idx],
                            want_rad,
                            color = "red",
                            linewidth = 0.5,
                            label = "Want Rad"
                        )
                        axes.legend(loc = "upper left", bbox_to_anchor = (0, -0.1))
                        axes.set_xlabel("Wavelength")
                        axes.set_ylabel("Rad")
                        fig.suptitle("")   
                        fig.tight_layout()
                        plt.show()

                    c0 = np.copy(w0) * 0
                    a, b = a0, b0
                    for k in np.arange(nch1, nch2 - 1):
                        cref0 = (
                            ((a+b*cref)  * np.exp((w0[k] - wref) * b) - a)
                            / b
                        )
                        cfit, _ = get_channel(wavelength_model, irradiance_model_smooth, w0[k], dw, chn, rad, a, b, cref0)
                        c0[k] = cfit
                        idx = np.where(abs(wavelength_model - w0[k]) < dw)
                        want_model = (
                            (irradiance_model[idx] - np.min(irradiance_model[idx]))
                            / (np.max(irradiance_model[idx]) - np.min(irradiance_model[idx]))
                        )
                        smooth_model = (
                            (irradiance_model_smooth[idx] - np.min(irradiance_model_smooth[idx]) )
                            / (np.max(irradiance_model_smooth[idx]) - np.min(irradiance_model_smooth[idx])) 
                        )
                        # Plot 4
                        if showPlots:
                            fig, axes = plt.subplots(1, 1, figsize = (2.5 * fs, fs))
                            axes.plot(
                                wavelength_model[idx],
                                want_model,
                                color = "blue",
                                linewidth = 0.5,
                                label = "Model"
                            )
                            axes.plot(
                                wavelength_model[idx],
                                smooth_model,
                                color = "orange",
                                linewidth = 0.5,
                                label = "Smooth Model"
                            )
                        wl = w0[k] + np.log((a + b*chn) / (a + b*cfit)) / b
                        idx = np.where(abs(wl - w0[k]) < dw)
                        want_rad = (
                            (rad[idx] - np.min(rad[idx]))
                            / (np.max(rad[idx]) - np.min(rad[idx]))
                        )
                        if showPlots:
                            axes.plot(
                                wl[idx],
                                want_rad,
                                color = "red",
                                linewidth = 0.5,
                                label = "Want Rad"
                            )
                            axes.set_xlabel("Wavelength (nm)")
                            axes.set_ylabel("Intensity")
                            fig.suptitle("")   
                            fig.tight_layout()
                            plt.show()

                    mm = 10
                    tmax = 0
                    all_res = []
                    for iter_i in tqdm(range(-mm, mm), desc = "Iterating through log-linear model to compute a and b"):
                        for iter_j in range(-mm, mm):
                            a = a0 *(1 + 0.01*iter_i)
                            b = b0 *(1 + 0.01*iter_j)
                            amax = 1
                            for iter_k in range(nch1, nch2 - 1):
                                cfit, cmax = get_channel(wavelength_model, irradiance_model_smooth, w0[iter_k], dw, chn, rad, a, b, c0[iter_k])
                                amax = amax * cmax
                            if amax > tmax:
                                # print(w0[iter_k], cfit, tmax, " -> ", amax)
                                tmax = amax
                                afit = a    
                                bfit = b
                            
                                # print(iter_i, iter_j, afit, bfit, tmax)
                                all_res.append({"i" : iter_i, "j" : iter_j, "a" : afit, "b" : bfit, "corr" : tmax})
                    all_res = pd.DataFrame(all_res)
                    best_idx = np.argmax(all_res["corr"])
                    print(f"New best guess:\na0 = {all_res["a"][best_idx]}\nb0 = {all_res["b"][best_idx]}\ncorr = {all_res["corr"][best_idx]}")
                    # a=afit & b=bfit
                    # wl=wref+alog((a+b*chn)/(a+b*cref))/b	; log-linear model

                    for nfit in range(1, 2+1):
                        # Plot 5
                        fig, axes = plt.subplots(1, 1, figsize = (fs, fs))
                        nonzero_idx = np.where(c0 != 0)
                        num_nonzero_idx = len(nonzero_idx)
                        axes.plot(
                            c0[nonzero_idx],
                            w0[nonzero_idx],
                            label = "Poly Fit Model",
                            color = "blue",
                            ls = "",
                            marker = "D",
                            markersize = 1
                        )
                        axes.set_title("Polyfit Model")
                        axes.legend(loc = "upper left", bbox_to_anchor = (0, -.15))
                        axes.set_xlabel("Channel")
                        axes.set_ylabel("Wavelength")
                        # axes.set_box_aspect(1)

                        x = c0[nonzero_idx]
                        # x = np.zeros((nfit, num_nonzero_idx))
                        # for it in range(0, nfit-1):
                            # x[it, :] = c0[nonzero_idx]**(it+1)

                        cc = np.polyfit(x, w0[nonzero_idx], nfit)
                        cc0 = cc[-1]
                        yfit = np.polyval(cc, x)
                        cc = cc[:-1]

                        axes.plot(
                            x,
                            yfit,
                            color = "red",
                            linewidth = 0.5,
                            alpha = 0.5
                        )

                        wl = cc0
                        for iter_i in range(0, nfit):
                            wl = wl + cc[iter_i] * chn**(iter_i + 1)
                        wl_poly = wl 
                        axes.plot(
                            chn,
                            wl,
                            color = "green",
                            alpha = 0.5,
                            ls = '--'
                        )

                        fig.suptitle("")   
                        fig.tight_layout()
                        plt.show()

                        want_model = irradiance_model / np.max(irradiance_model)
                        fig, axes = plt.subplots(1, 1, figsize = (2.5 * fs, fs))
                        smooth_model = irradiance_model_smooth / np.max(irradiance_model_smooth)
                        want_rad = rad * 1 / np.max(rad)
                        axes.plot(
                            wavelength_model,
                            want_model,
                            color = "blue",
                            linewidth = 0.5
                        )
                        axes.plot(
                            wavelength_model,
                            smooth_model,
                            color = "orange",
                            linewidth = 0.5
                        )
                        axes.plot(
                            wl_poly,
                            want_rad,
                            color = "red",
                            linewidth = 0.5
                        )
                        ylim = axes.get_ylim()
                        axes.vlines(
                            wref,
                            -1e6, 
                            1e10,
                            color = "black",
                            ls = "--",
                            alpha = 0.5
                        )
                        axes.set_ylim(ylim)
                        fig.tight_layout()
                        plt.show()

                        return wl_poly
                    


data_input_path = r"C:\GSFC\2025\05 May\Pandora2s1_GreenbeltMD_20250505_L0.txt"
# data_input_path = r"C:\GSFC\Pandora253s1_TucsonAZ_20250520_L0.txt"
model_input_path = r"C:\GSFC\model_01292025_Reptranfiner0.1.txt"
x_fit = fitSpectrum(model_input_path, data_input_path, fs = 3, showPlots = True)


In [ ]:
in_data = get_important_info(data_input_path, dark_corrected = True)

In [ ]:
dataset2obs = range(len(in_data[0]))
plt.close('all')
for d in dataset2obs[:1]:
    dataset1 = in_data[0][d]
    dataset2 = in_data[0][d + 1]
    rad = dataset1 - dataset2
    fs = 3
    fig, axes = plt.subplots(1, 1, figsize = (2.5 * fs, fs))
    axes.plot(
        x_fit,
        rad
    )
    axes.set_xlabel("Wavelength (nm)")
    axes.set_ylabel(r"$\frac{W}{nm\quadm^2}$", size = 12)
    axes.set_title(f"{in_data[-1][d]}")
    fig.tight_layout()
    plt.show()


In [ ]:
# # Wu/ChatGPT spec_cal_test.pro -> python
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy.interpolate import interp1d
# from scipy.optimize import curve_fit
# data_input_path = r"C:\GSFC\2025\05 May\Pandora2s1_GreenbeltMD_20250505_L0.txt"
# model_input_path = r"model_01292025_Reptranfiner0.1.txt"

# def smooth(data, window):
#     return np.convolve(data, np.ones(window)/window, mode='same')

# def get_chn(wl_model, irr_model, wref, dw, chn, rad, a, b, cref0):
#     cmax = 0
#     nn = 100  # Search range

#     ii = np.where(np.abs(wl_model - wref) < dw)[0]
#     wl_m = wl_model[ii]
#     want_model = (irr_model[ii] - np.min(irr_model[ii])) / (np.max(irr_model[ii]) - np.min(irr_model[ii]))

#     cfit = cref0
#     for k in range(-nn, nn + 1):
#         cref = cref0 + 0.5 * k  
#         wl = wref + np.log((a + b * chn) / (a + b * cref)) / b
#         ii = np.where(np.abs(wl - wref) < dw)[0]

#         if len(ii) == 0:
#             continue

#         want_rad = (rad[ii] - np.min(rad[ii])) / (np.max(rad[ii]) - np.min(rad[ii]))
#         interp_func = interp1d(wl[ii], want_rad, fill_value="extrapolate")
#         want = interp_func(wl_m)

#         corr = np.corrcoef(want_model, want)[0, 1]

#         if corr > cmax:
#             cmax = corr
#             cfit = cref

#     return cfit, cmax

# def fit_spectrum(modpath):
#     wl_model, irr_model = get_model(modpath)
#     icut = 5
#     irr_model_smooth = smooth(irr_model, icut)

#     plt.figure(figsize=(8, 4))
#     plt.plot(wl_model, irr_model, label="Model Spectrum", color='black', linewidth = 0.5)
#     plt.plot(wl_model, irr_model_smooth, label=f"Smoothed ({icut})", color='blue', linewidth=0.5)
#     plt.xlabel("Wavelength (nm)")
#     plt.ylabel("Intensity")
#     plt.legend()
#     plt.title("Model Spectrum Fitting")
#     plt.show()

#     chn = np.arange(2048)
#     w0 = np.array([822.7, 759.4, 722.2, 686.7, 656.3, 627.7, 589.3, 527.0,
#                    518.4, 495.8, 486.1, 438.6, 430.8, 410.2, 396.8, 382.0, 358.1, 336.1])

#     dw = 10
#     a0, b0 = 6.3, 0.0013
#     wref, cref0 = 430.8, 1150
    
#     cfit, cmax = get_chn(wl_model, irr_model_smooth, wref, dw, chn, irr_model, a0, b0, cref0)

#     plt.figure(figsize=(8, 4))
#     plt.plot(wl_model, irr_model, label="Model Spectrum", linewidth = 0.5)
#     plt.axvline(wref, color="red", linestyle="--", label="Reference Wavelength")
#     plt.axvline(wl_model[int(cfit)], color="green", linestyle="--", label="Best Fit Channel")
#     plt.legend()
#     plt.xlabel("Wavelength (nm)")
#     plt.ylabel("Intensity")
#     plt.title("Best Fit Channel")
#     plt.show()

# fit_spectrum(model_input_path) 